# 08 - Python Finance

**Capitulo 08**: Análise de empresas com Python, obtendo dados das empresas listadas na bolsa usando Python.

Realizar webscraping (raspagem de dados) e baixar os dados dos balanços das empresas disponíveis no site **fundamentus**.

Em seguida criar um ranking das melhores empresas segundo a da Magic Formula de **Joel Greenblatt**.

## **Configurações Iniciais**

## 1. Importando bibliotecas

In [1]:
import numpy as np
import pandas as pd
import string
import warnings
warnings.filterwarnings('ignore')

import requests

## 2. Obtendo e tratando os dados

In [2]:
url = 'http://www.fundamentus.com.br/resultado.php'

In [3]:
header = {
  "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36"  
 }

r = requests.get(url, headers=header)

In [4]:
df = pd.read_html(r.text,  decimal=',', thousands='.')[0]

Exibindo dados

In [5]:
df

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
0,ODER3,0.00,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,0.00,0.00,"0,00%","0,00%",0.00,"0,00%","41,79%",0.0,2.842660e+08,0.00,"10,81%"
1,CSTB4,147.69,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,0.00,0.00,"40,85%","28,98%",2.60,"22,40%","20,11%",0.0,8.420670e+09,0.14,"31,91%"
2,EQMA5B,0.00,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,0.00,0.00,"0,00%","0,00%",0.00,"0,00%","23,36%",0.0,2.997240e+09,0.00,"7,29%"
3,MNSA3,0.42,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,0.00,0.00,"-208,15%","-362,66%",3.63,"-13,50%","145,70%",0.0,-9.105000e+06,-6.52,"-41,11%"
4,CLSC6,0.00,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,0.00,0.00,"0,00%","0,00%",0.00,"0,00%","26,13%",0.0,1.984640e+09,0.00,"9,05%"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
936,LWSA3,23.00,682.15,19.98,27.582,"0,12%",9.272,37.73,311.15,462.67,304.88,132.11,"8,86%","4,04%",1.79,"4,30%","2,93%",270431000.0,6.739730e+08,0.23,"24,50%"
937,PRBC4,14.54,808.37,39.12,0.000,"0,00%",0.000,0.00,0.00,0.00,0.00,0.00,"0,00%","0,00%",0.00,"0,00%","4,84%",0.0,1.210880e+09,0.00,"11,60%"
938,UBBR11,14.75,1201.81,3.91,0.000,"0,00%",0.000,0.00,0.00,0.00,0.00,0.00,"0,00%","0,00%",0.00,"0,00%","0,33%",0.0,1.031720e+10,0.00,"10,58%"
939,UBBR3,18.00,1466.61,4.77,0.000,"0,00%",0.000,0.00,0.00,0.00,0.00,0.00,"0,00%","0,00%",0.00,"0,00%","0,33%",0.0,1.031720e+10,0.00,"10,58%"


In [6]:
for coluna in ['Div.Yield', 'Mrg Ebit', 'Mrg. Líq.', 'ROIC', 'ROE', 'Cresc. Rec.5a']:
  df[coluna] = df[coluna].str.replace('.', '')
  df[coluna] = df[coluna].str.replace(',', '.')
  df[coluna] = df[coluna].str.rstrip('%').astype('float') / 100

Exibindo dados tratados

In [7]:
df

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
0,ODER3,0.00,0.00,0.00,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0.00,0.0000,0.4179,0.0,2.842660e+08,0.00,0.1081
1,CSTB4,147.69,0.00,0.00,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,0.00,0.4085,0.2898,2.60,0.2240,0.2011,0.0,8.420670e+09,0.14,0.3191
2,EQMA5B,0.00,0.00,0.00,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0.00,0.0000,0.2336,0.0,2.997240e+09,0.00,0.0729
3,MNSA3,0.42,0.00,0.00,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,0.00,-2.0815,-3.6266,3.63,-0.1350,1.4570,0.0,-9.105000e+06,-6.52,-0.4111
4,CLSC6,0.00,0.00,0.00,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0.00,0.0000,0.2613,0.0,1.984640e+09,0.00,0.0905
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
936,LWSA3,23.00,682.15,19.98,27.582,0.0012,9.272,37.73,311.15,462.67,304.88,132.11,0.0886,0.0404,1.79,0.0430,0.0293,270431000.0,6.739730e+08,0.23,0.2450
937,PRBC4,14.54,808.37,39.12,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0.00,0.0000,0.0484,0.0,1.210880e+09,0.00,0.1160
938,UBBR11,14.75,1201.81,3.91,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0.00,0.0000,0.0033,0.0,1.031720e+10,0.00,0.1058
939,UBBR3,18.00,1466.61,4.77,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0.00,0.0000,0.0033,0.0,1.031720e+10,0.00,0.1058


## 2. Analisando os dados coletados


In [8]:
df.shape

(941, 21)

Liquidez diaria dos últimos dois meses maior 1k, empresas com negociação média superior a 1k por dia nos últimos dois meses.

In [9]:
df = df[df['Liq.2meses'] > 1000000]

Excluindo empresas pequenas com baixa negociação da análise.

In [10]:
df.shape

(247, 21)

Listar empresas segundo os critérios, **'pos', 'EV/EBIT', 'ROIC'**

In [11]:
ranking = pd.DataFrame()
ranking['pos'] = range(1,151)
ranking['EV/EBIT'] = df[ df['EV/EBIT'] > 0 ].sort_values(by=['EV/EBIT'])['Papel'][:150].values
ranking['ROIC'] = df.sort_values(by=['ROIC'], ascending=False)['Papel'][:150].values #maior para menor

Exibindo dados ranqueados

In [12]:
ranking

,pos,EV/EBIT,ROIC
0,1,PSSA3,WIZS3
1,2,PCAR3,PSSA3
2,3,WIZS3,ODPV3
3,4,MRFG3,MOSI3
4,5,ALUP11,CMIN3
...,...,...,...
145,146,RENT3,RLOG3
146,147,CSAN3,DMVF3
147,148,ELET3,PRIO3
148,149,ELET6,RAIL3


Formula magica somar os dois rankings, quais empresas tem os melhores valores (empresas melhores classificadas com relação ao preço e relação a qualidade).

In [15]:
a = ranking.pivot_table(columns='EV/EBIT', values='pos')

In [16]:
b = ranking.pivot_table(columns='ROIC', values='pos')

In [17]:
t = pd.concat([a,b])
t

,ABEV3,AGRO3,ALSO3,ALUP11,AMBP3,ANIM3,ASAI3,AURA33,B3SA3,BEEF3,BRDT3,BRFS3,BRKM5,BRML3,BRPR3,BSEV3,CAML3,CARD3,CCPR3,CCRO3,CESP6,CIEL3,CMIG3,CMIG4,CMIN3,COCE5,CPFE3,CPLE3,CPLE6,CRFB3,CSAN3,CSMG3,CSNA3,CURY3,CYRE3,DIRR3,DMVF3,DTEX3,ECOR3,EGIE3,...,TEND3,TESA3,TGMA3,TIET11,TIET4,TIMS3,TOTS3,TRIS3,TRPL4,TUPY3,UGPA3,UNIP6,USIM3,USIM5,VALE3,VAMO3,VIVA3,VIVT3,VLID3,VVAR3,WIZS3,WSON33,AERI3,ALPA4,BOAS3,GNDI3,HAPV3,LINX3,LREN3,MGLU3,MOSI3,PETZ3,PNVL3,PRIO3,RADL3,RDOR3,SULA11,TFCO4,WEGE3,YDUQ3
pos,102,81,103.0,5,121,99,53,50,110,15,115,72,44,127.0,136.0,59,52,70,76,118,104.0,131.0,29,17,24,107.0,58,16,20,63,147,26,49,69,122,48,23,109,47,51,...,36,43,80,18,19,56,93,67,9,83,133,42,57,55,6,144,141,90,130.0,60,3,25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pos,38,93,NaN,36,58,129,17,11,24,20,124,92,60,NaN,NaN,33,54,37,145,119,NaN,NaN,76,77,5,NaN,52,88,87,48,140,62,23,21,142,138,147,99,30,29,...,59,101,50,16,15,100,14,81,31,126,136,18,85,84,13,61,55,134,NaN,64,1,34,72.0,41.0,74.0,67.0,71.0,127.0,106.0,141.0,4.0,75.0,131.0,148.0,95.0,133.0,105.0,28.0,19.0,150.0


In [18]:
# descartar as empresas que não tem os dois rank, indicadores EV/EBIT e ROIC
rank = t.dropna(axis=1).sum()
rank

ABEV3     140
AGRO3     174
ALUP11     41
AMBP3     179
ANIM3     228
         ... 
VIVA3     196
VIVT3     224
VVAR3     124
WIZS3       4
WSON33     59
Length: 132, dtype: int64

## 3. Resultados


In [19]:
rank.sort_values()[:38]

PSSA3      3
WIZS3      4
MRFG3     13
PLPL3     14
VALE3     19
CMIN3     29
TIET4     34
TIET11    34
BEEF3     35
TASA4     38
TRPL4     40
TASA3     40
ALUP11    41
PCAR3     53
SAPR11    55
SAPR4     56
JBSS3     56
SAPR3     57
LEVE3     57
WSON33    59
UNIP6     60
AURA33    61
TAEE3     62
TAEE11    64
TAEE4     67
LAVV3     68
ASAI3     70
CSNA3     72
EQTL3     73
ODPV3     74
ECOR3     77
EGIE3     80
CSMG3     88
QUAL3     89
CURY3     90
BSEV3     92
CMIG4     94
TEND3     95
dtype: int64

Remover empresas seguradoras, empresas de bancos.

Distorções quando comparadas empresas do setor financeiro utilizando os multiplos (indicadores EV/EBIT e ROIC).

## 4. Observações

**Empresas baratas Brasileiras**

*Empresas que estejam valendo pouco comparadas ao tanto que ela gera de caixa.*



O EV/EBIT é um indicador financeiro que compara o Valor da Firma (EV ou Enterprise Value) com o Lucro Antes de Impostos e Taxas, o EBIT.

A principal função do EV/EBIT é ajudar a identificar quanto uma empresa custa em relação ao que ela produz a partir de sua atividade fim.

**EV/EBIT**

EV ---> Valor da Firma
Valor da Firma ---> Valor de mercado + Dív. Líquida

*Valor de mercado ---> Valor Ação (Cotação ou Preço atual) * Nro. de Ações emitidas*

EBIT ---> Lucro Líquido + Resultado Financeiro + Impostos

EV/EBIT ---> Valor de mercado (EV) / Lucro Antes de Juros e Impostos (EBIT)




É importante destacar que na fórmula EV/EBIT, o EV representa o valor de mercado da empresa somado à sua dívida líquida. Enquanto isso, o EBIT representa o nível de lucro operacional que a empresa produz em seu tamanho atual.

Nesse sentido, o EV/EBIT permite saber qual o potencial de geração de lucros de uma companhia. Vale lembrar que, para fins comparativos, ele deve ser utilizado apenas entre empresas do mesmo setor e, de preferência, que sejam concorrentes diretas.

**Exemplo:**

Uma empresa “A” e uma empresa “B” possuem, respectivamente, um EV/EBIT de 9,2 vezes e 8,9 vezes, seria possível afirmar que:

A empresa “A” está mais valorizada e melhor avaliada no mercado, podendo ser a melhor opção para a venda;

A empresa “B” está mais desvalorizada e pior avaliada no mercado, podendo ser a melhor opção para a compra.

Quando o EV/EBIT está elevado, existe uma indicação de que a empresa possui uma boa avaliação no mercado. Consequentemente, isso significa que suas ações estão valorizadas.

Enquanto isso, um EV/EBIT baixo demonstra que a empresa está sendo subavaliada. Com isso, possuem uma forte tendência de valorização ao longo do tempo, tornando a ação mais atrativa para a compra.

**Empresas boas Brasileiras**

O ROIC (Return Over Invested Capital), ou Retorno sobre o Capital Investido, é uma métrica utilizada para informar, em termos percentuais, quanto dinheiro uma empresa consegue gerar em razão do capital investido.

Com o ROIC, é possível ter uma visão geral sobre a performance financeira de um negócio.

**ROIC**

NOPAT ---> representa o lucro operacional após a distribuição de dividendos entre os acionistas

Valor Contábil ---> é a soma de todo capital investido na empresa.

ROIC ---> NOPAT / Valor Contábil

**Exemplo de uma empresa:**

Lucro Operacional: R$20 milhões

Valor Contábil do Capital Investido: R$80 milhões

Dividendos: R$5 milhões

NOPAT dessa empresa é igual a R$15 milhões

ROIC ---> 15.000.000 / 80.000.000 = 18,75%

(Lucro Operacional - Dividendos) / Valor Contábil do Capital Investido = ROIC



O parâmetro para avaliar se esse percentual é elevado ou não será o histórico da empresa e de suas concorrentes do mesmo setor.

Vale ressaltar que esse é um dado bem relevante pois demonstra o retorno sobre capital investido na empresa, mostrando muitas vezes qual companhia pode utilizar de forma mais eficiente o capital captado através de empréstimos, financiamentos ou acionistas.

Por conta dessas características, o ROIC é um dos indicadores financeiros mais utilizados por investidores na análise fundamentalista de ações.

Pois, muitos investidores acabam utilizando esse indicador na decisão sobre qual empresa realizar um investimento.